In [ ]:
import cv2
from scipy.spatial import distance as dist
from imutils.video import VideoStream
import numpy as np
import imutils
import time

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

# Load the pre-trained face detection model
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Open a connection to the camera (0 represents the default camera)
vs = VideoStream(src=0).start()
time.sleep(1.0)

blink_counter = 0
blink_threshold = 0.9 # Adjust this threshold based on your observations

while True:
    # Capture frame-by-frame
    frame = vs.read()

    # Convert the frame to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Extract the region of interest for eye blink detection
        roi_gray = gray[y:y + h, x:x + w]

        # Extract the left and right eye coordinates
        leftEye = roi_gray[5:11, 0:w]
        rightEye = roi_gray[5:11, w - 6:w]

        leftEAR = eye_aspect_ratio(leftEye)
        rightEAR = eye_aspect_ratio(rightEye)

        ear = (leftEAR + rightEAR) / 2.0

        # Check for blinks
        if ear < blink_threshold:
            blink_counter += 1
            cv2.putText(frame, f"Blink Count: {blink_counter}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Calculate the average intensity within the eye regions
        average_intensity_left = np.mean(leftEye)
        average_intensity_right = np.mean(rightEye)

        # Define a threshold for the intensity change
        intensity_threshold = 12

        # Check for sudden changes in intensity
        if abs(average_intensity_left - average_intensity_right) > intensity_threshold:
            cv2.putText(frame, "Awake", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        else:
            cv2.putText(frame, "Sleepy", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        # Draw contours for the eyes inside the detected face region
        leftEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))
        rightEyeHull = cv2.convexHull(np.array([(30, 120), (30, 60), (w-30, 120), (w-30, 60)]))

        cv2.drawContours(frame[y:y + h, x:x + w], [leftEyeHull], -1, (0, 255, 0), 2)
        cv2.drawContours(frame[y:y + h, x:x + w], [rightEyeHull], -1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Face and Eye Detection', frame)

    # Break the loop if 'q' is pressed
    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break

# Release the camera and close the window
vs.stop()
cv2.destroyAllWindows()
